In [1]:
import pandas as pd
import re

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
#Read File Meta Reviews and Geometry
dataframe_RG=pd.read_csv("RG_Data.csv")
dataframe_RG.head(2)

,placekey,Month,Week Start,spend_by_transaction_intermediary,online_spend,online_transactions,Sum_spent_by_week,Average_spent_per_transaction_by_week,location_name,street_address,city,region,postal_code,latitude,longitude
0,222-222@39p-8k2-mtv,['2022-07-01'],"['2022-06-27', '2022-07-04', '2022-07-11', '20...",{'No intermediary'},0,0,"[0.0, 202.9, 61.65, 0.0, 0.0]","[0.0, 28.99, 8.81, 0.0, 0.0]",United States Postal Service (USPS),100 6th Ave,Kaltag,AK,99748,64.324568,-158.723724
1,222-222@39s-pzs-wrk,['2019-07-01'],"['2019-07-01', '2019-07-08', '2019-07-15', '20...",{'No intermediary'},0,0,"[0.0, 47.5, 0.0, 0.0, 0.0]","[0.0, 3.39, 0.0, 0.0, 0.0]",United States Postal Service (USPS),123 Takotna Ave,Mc Grath,AK,99627,62.951525,-155.591123


### Check Unique gmap_ids , placekeys and count of rows

In [4]:
dataframe_RG.shape[0]

734442

In [5]:
len(dataframe_RG["placekey"].unique())

734442

### Expand Revenues_and_Geometry

In [6]:
total_3=dataframe_RG[["placekey",'Month', 'spend_by_transaction_intermediary','online_spend', 'online_transactions', 
    'Week Start','Sum_spent_by_week', 'Average_spent_per_transaction_by_week']]

In [7]:
#Month
total_3.loc[:,'Month']=total_3['Month'].apply(lambda x:x.strip('][').replace("'","").split(','))
total_3.loc[:,'Month']=total_3['Month'].apply(lambda x:[temp.strip() for temp in x])

In [8]:
#Week_Start
total_3.loc[:,'Week Start']=total_3['Week Start'].apply(lambda x:x.strip('][').replace("'","").split(','))
total_3["Week Start"]=total_3["Week Start"].apply(lambda x:[temp.strip() for temp in x])

In [9]:
#Visits Weekly Level
total_3.loc[:,'Sum_spent_by_week']=total_3['Sum_spent_by_week'].apply(lambda x:x.strip('][').replace("'","").split(','))
total_3.loc[:,'Sum_spent_by_week']=total_3['Sum_spent_by_week'].apply(lambda x:[float(temp) for temp in x])
total_3.loc[:,'Sum_spent_by_week']=total_3['Sum_spent_by_week'].apply(lambda x:[round(temp,2) for temp in x])

In [10]:
#Average Spent By week
total_3.loc[:,'Average_spent_per_transaction_by_week']=total_3['Average_spent_per_transaction_by_week'].apply(lambda x:x.strip('][').replace("'","").split(','))
total_3.loc[:,'Average_spent_per_transaction_by_week']=total_3['Average_spent_per_transaction_by_week'].apply(lambda x:[float(temp) for temp in x])
total_3.loc[:,'Average_spent_per_transaction_by_week']=total_3['Average_spent_per_transaction_by_week'].apply(lambda x:[round(temp,2) for temp in x])

### 1.a Weekly Level

In [11]:
total_3_a=total_3[["placekey","Week Start","Sum_spent_by_week","Average_spent_per_transaction_by_week"]]
total_3_a=total_3_a.explode(["Week Start","Sum_spent_by_week","Average_spent_per_transaction_by_week"])
total_3_a.head()

,placekey,Week Start,Sum_spent_by_week,Average_spent_per_transaction_by_week
0,222-222@39p-8k2-mtv,2022-06-27,0.0,0.0
0,222-222@39p-8k2-mtv,2022-07-04,202.9,28.99
0,222-222@39p-8k2-mtv,2022-07-11,61.65,8.81
0,222-222@39p-8k2-mtv,2022-07-18,0.0,0.0
0,222-222@39p-8k2-mtv,2022-07-25,0.0,0.0


In [12]:
total_3_a["Month_Match"]=total_3_a["Week Start"].apply(lambda x:x[0:7]+"-01")
total_3_a

,placekey,Week Start,Sum_spent_by_week,Average_spent_per_transaction_by_week,Month_Match
0,222-222@39p-8k2-mtv,2022-06-27,0.0,0.0,2022-06-01
0,222-222@39p-8k2-mtv,2022-07-04,202.9,28.99,2022-07-01
0,222-222@39p-8k2-mtv,2022-07-11,61.65,8.81,2022-07-01
0,222-222@39p-8k2-mtv,2022-07-18,0.0,0.0,2022-07-01
0,222-222@39p-8k2-mtv,2022-07-25,0.0,0.0,2022-07-01
...,...,...,...,...,...
734441,zzy-224@96z-yp4-qvf,2022-08-01,0.0,0.0,2022-08-01
734441,zzy-224@96z-yp4-qvf,2022-08-08,0.0,0.0,2022-08-01
734441,zzy-224@96z-yp4-qvf,2022-08-15,0.0,0.0,2022-08-01
734441,zzy-224@96z-yp4-qvf,2022-08-22,16.03,1.15,2022-08-01


### 2.b Monthly Level

In [13]:
total_3_b=total_3[["placekey",'Month','online_spend', 'online_transactions']]
total_3_b.head()

,placekey,Month,online_spend,online_transactions
0,222-222@39p-8k2-mtv,[2022-07-01],0,0
1,222-222@39s-pzs-wrk,[2019-07-01],0,0
2,222-222@39x-s8g-yy9,[2022-07-01],0,0
3,222-222@39x-sn7-y35,[2022-03-01],0,0
4,222-222@39y-p4c-ndv,[2022-02-01],0,0


In [14]:
total_3_b_a=total_3_b[total_3_b["online_spend"]=="0"].explode(["Month"])
total_3_b_a.head()

,placekey,Month,online_spend,online_transactions
0,222-222@39p-8k2-mtv,2022-07-01,0,0
1,222-222@39s-pzs-wrk,2019-07-01,0,0
2,222-222@39x-s8g-yy9,2022-07-01,0,0
3,222-222@39x-sn7-y35,2022-03-01,0,0
4,222-222@39y-p4c-ndv,2022-02-01,0,0


In [15]:
total_3_b_b=total_3_b[total_3_b["online_spend"]!="0"]

In [16]:
#Online Spend
total_3_b_b.loc[:,'online_spend']=total_3_b_b['online_spend'].apply(lambda x:x.strip('][').replace("'","").split(','))
total_3_b_b.loc[:,'online_spend']=total_3_b_b['online_spend'].apply(lambda x:[float(temp) for temp in x])
total_3_b_b.loc[:,'online_spend']=total_3_b_b['online_spend'].apply(lambda x:[round(temp,2) for temp in x])

In [17]:
#Online Transactions
total_3_b_b.loc[:,'online_transactions']=total_3_b_b['online_transactions'].apply(lambda x:x.strip('][').replace("'","").split(','))
total_3_b_b.loc[:,'online_transactions']=total_3_b_b['online_transactions'].apply(lambda x:[float(temp) for temp in x])
total_3_b_b.loc[:,'online_transactions']=total_3_b_b['online_transactions'].apply(lambda x:[round(temp,2) for temp in x])

In [18]:
total_3_b_b["Count_1"]=total_3_b_b["online_transactions"].apply(lambda x:len(x))
total_3_b_b["Count_2"]=total_3_b_b["Month"].apply(lambda x:len(x))

In [19]:
total_3_b_b_1=total_3_b_b[total_3_b_b["Count_1"]==total_3_b_b["Count_2"]]
total_3_b_b_1.drop(columns=["Count_1","Count_2"],inplace=True)

In [20]:
total_3_b_b_1=total_3_b_b_1.explode(["Month","online_spend","online_transactions"])

In [21]:
total_3_b_b_1

,placekey,Month,online_spend,online_transactions
25,222-222@3bt-c2f-sh5,2022-07-01,2125.0,5.0
33,222-222@3f4-s9g-kcq,2020-03-01,254.8,3.0
55,222-222@3x5-4k5-3bk,2021-08-01,200.0,2.0
64,222-222@3x5-4sd-d5f,2021-12-01,423.62,1.0
75,222-222@3x5-6pd-vs5,2020-07-01,215.0,2.0
...,...,...,...,...
734441,zzy-224@96z-yp4-qvf,2022-04-01,0.0,0.0
734441,zzy-224@96z-yp4-qvf,2022-05-01,0.0,0.0
734441,zzy-224@96z-yp4-qvf,2022-06-01,0.0,0.0
734441,zzy-224@96z-yp4-qvf,2022-07-01,7.63,0.5


In [22]:
total_3_b=pd.concat([total_3_b_a,total_3_b_b_1])
total_3_b.head()

,placekey,Month,online_spend,online_transactions
0,222-222@39p-8k2-mtv,2022-07-01,0,0
1,222-222@39s-pzs-wrk,2019-07-01,0,0
2,222-222@39x-s8g-yy9,2022-07-01,0,0
3,222-222@39x-sn7-y35,2022-03-01,0,0
4,222-222@39y-p4c-ndv,2022-02-01,0,0


### 2.c Spend Transaction Intermediary

In [23]:
total_3_c=total_3[["placekey",'Month', 'spend_by_transaction_intermediary']]
total_3_c.head()

,placekey,Month,spend_by_transaction_intermediary
0,222-222@39p-8k2-mtv,[2022-07-01],{'No intermediary'}
1,222-222@39s-pzs-wrk,[2019-07-01],{'No intermediary'}
2,222-222@39x-s8g-yy9,[2022-07-01],{'No intermediary'}
3,222-222@39x-sn7-y35,[2022-03-01],{'No intermediary'}
4,222-222@39y-p4c-ndv,[2022-02-01],{'No intermediary'}


In [24]:
total_3_c_a=total_3_c.loc[~total_3_c["spend_by_transaction_intermediary"].str.contains("]")].explode("Month")
total_3_c_a

,placekey,Month,spend_by_transaction_intermediary
0,222-222@39p-8k2-mtv,2022-07-01,{'No intermediary'}
1,222-222@39s-pzs-wrk,2019-07-01,{'No intermediary'}
2,222-222@39x-s8g-yy9,2022-07-01,{'No intermediary'}
3,222-222@39x-sn7-y35,2022-03-01,{'No intermediary'}
4,222-222@39y-p4c-ndv,2022-02-01,{'No intermediary'}
...,...,...,...
734430,22g-222@8gh-x7c-p5f,2022-04-01,{'No intermediary'}
734430,22g-222@8gh-x7c-p5f,2022-05-01,{'No intermediary'}
734430,22g-222@8gh-x7c-p5f,2022-06-01,{'No intermediary'}
734430,22g-222@8gh-x7c-p5f,2022-07-01,{'No intermediary'}


In [25]:
total_3_c_b=total_3_c.loc[total_3_c["spend_by_transaction_intermediary"].str.contains("]")]
total_3_c_b["spend_by_transaction_intermediary"]=total_3_c_b["spend_by_transaction_intermediary"].apply(lambda x:eval(x))
total_3_c_b=total_3_c_b.explode(["Month","spend_by_transaction_intermediary"])

In [26]:
total_3_c=pd.concat([total_3_c_a,total_3_c_b])
total_3_c.head()

,placekey,Month,spend_by_transaction_intermediary
0,222-222@39p-8k2-mtv,2022-07-01,{'No intermediary'}
1,222-222@39s-pzs-wrk,2019-07-01,{'No intermediary'}
2,222-222@39x-s8g-yy9,2022-07-01,{'No intermediary'}
3,222-222@39x-sn7-y35,2022-03-01,{'No intermediary'}
4,222-222@39y-p4c-ndv,2022-02-01,{'No intermediary'}


In [27]:
#First Merge
total_3_2=pd.merge(total_3_b,total_3_c,on=["placekey","Month"])
total_3_2

,placekey,Month,online_spend,online_transactions,spend_by_transaction_intermediary
0,222-222@39p-8k2-mtv,2022-07-01,0,0,{'No intermediary'}
1,222-222@39s-pzs-wrk,2019-07-01,0,0,{'No intermediary'}
2,222-222@39x-s8g-yy9,2022-07-01,0,0,{'No intermediary'}
3,222-222@39x-sn7-y35,2022-03-01,0,0,{'No intermediary'}
4,222-222@39y-p4c-ndv,2022-02-01,0,0,{'No intermediary'}
...,...,...,...,...,...
17263579,zzy-224@96z-yp4-qvf,2022-04-01,0.0,0.0,No intermediary
17263580,zzy-224@96z-yp4-qvf,2022-05-01,0.0,0.0,No intermediary
17263581,zzy-224@96z-yp4-qvf,2022-06-01,0.0,0.0,No intermediary
17263582,zzy-224@96z-yp4-qvf,2022-07-01,7.63,0.5,"{'No intermediary': 75.59, 'Visa': 15.26}"


### Total Output

In [28]:
total_3=pd.merge(total_3_a,total_3_2,left_on=["placekey","Month_Match"],right_on=["placekey","Month"],how="outer")

## Output

In [29]:
total_3.rename(columns={
    "Month_Match":"Month_RG",
    "Week Start":"Week Start_RG"
},inplace=True)

In [30]:
total_3.head()

,placekey,Week Start_RG,Sum_spent_by_week,Average_spent_per_transaction_by_week,Month_RG,Month,online_spend,online_transactions,spend_by_transaction_intermediary
0,222-222@39p-8k2-mtv,2022-06-27,0.0,0.0,2022-06-01,NaN,NaN,NaN,NaN
1,222-222@39p-8k2-mtv,2022-07-04,202.9,28.99,2022-07-01,2022-07-01,0,0,{'No intermediary'}
2,222-222@39p-8k2-mtv,2022-07-11,61.65,8.81,2022-07-01,2022-07-01,0,0,{'No intermediary'}
3,222-222@39p-8k2-mtv,2022-07-18,0.0,0.0,2022-07-01,2022-07-01,0,0,{'No intermediary'}
4,222-222@39p-8k2-mtv,2022-07-25,0.0,0.0,2022-07-01,2022-07-01,0,0,{'No intermediary'}


In [35]:
total_3=total_3[~total_3["Month_RG"].isna()]

In [36]:
#Revenues and Geometry
total_3["Month_RG"]=total_3["Month_RG"].apply(lambda x:x.strip())

In [37]:
total_3["Week Start_RG"]=total_3["Week Start_RG"].apply(lambda x:x.strip())

In [40]:
total_4=dataframe_RG[["placekey",'location_name','street_address', 'city', 'region', 'postal_code', 'latitude','longitude']]

In [41]:
total_4=pd.merge(total_3,total_4,on=["placekey"])

In [43]:
total_4[["placekey",'location_name', 'street_address', 'city', 'region','postal_code', 'latitude', 'longitude',
       'Month_RG', 'spend_by_transaction_intermediary','online_spend', 'online_transactions',
       'Week Start_RG','Sum_spent_by_week', 'Average_spent_per_transaction_by_week'
      ]].to_csv("RG_Decompressed.csv",index=False)